In [1]:
#!g1.1
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from IPython.display import clear_output

tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

model.to('cuda')
clear_output()

print('OK!')

OK!


In [8]:
#!g1.1
def generate(model, tokenizer, prompt, length=100, temperature=1., top_p=0.95, top_k=50, num_sequences=3, repetition_penalty=1.,
            **kwargs):
    input_ids = tokenizer.encode(prompt.strip(), add_special_tokens=False, return_tensors="pt").to('cuda')
    sample_outputs = model.generate(input_ids,
                            do_sample=True, 
                            max_length=length,
                            temperature=temperature,
                            top_k=top_k,
                            top_p=top_p,
                            repetition_penalty=repetition_penalty,
                            num_return_sequences=num_sequences, **kwargs)
    results = []
    for i, sample_output in enumerate(sample_outputs):
        results.append(tokenizer.decode(sample_output, skip_special_tokens=True))
    return results

In [29]:
#!g1.1

from tqdm.notebook import tqdm, trange

prompt = """
На ногах у Ивана резиновые сапоги. Иван ходит по лесу с ножом и корзиной. Это потому, что
"""

results = []

for i in trange(100):
    current = generate(model, tokenizer, prompt, num_sequences=10, temperature=0.7, repetition_penalty=1.5)
    for text in current:
        text = text[90:].split(".")[0].split("\n")[0]
        results.append(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [34]:
#!g1.1

from transformers import AutoTokenizer, AutoModel
import torch

sbert_tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
sbert_model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

In [49]:
#!g1.1

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embedding(sentences, batch_size=64):
    embeds = []
    for i in trange(0, len(sentences), batch_size):
        encoded_input = sbert_tokenizer(sentences[i:i + batch_size], padding=True, truncation=False, return_tensors='pt')
        with torch.inference_mode():
            model_output = sbert_model(**encoded_input.to("cuda"))
        sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
        embeds.append(sentence_embeddings)

    return torch.cat(embeds)

In [52]:
#!g1.1
embeddings = get_embedding(results).cpu()

In [54]:
#!g1.1
torch.save(embeddings, "data/test.pt")